# EfficientNet Model Repurposing - Experiments

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torchvision import models
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint


In [ ]:
data_dir = r"/exchange/dspro2/silent-speech/ASL_Dataset"
batch_size = 32
img_size = 224
num_workers = 64

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [64]:
train_dataset = datasets.ImageFolder(root=f"{data_dir}/Train", transform=data_transforms)
val_dataset = datasets.ImageFolder(root=f"{data_dir}/Test", transform=data_transforms)

In [65]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [66]:
class_names = train_dataset.classes
num_classes = len(class_names)

In [67]:
m = models.efficientnet_b0()

In [68]:
for param in m.classifier.parameters():
    if param.requires_grad:
        print(param.shape)

torch.Size([1000, 1280])
torch.Size([1000])


In [69]:
m = m.requires_grad_(False)
for param in m.classifier.parameters():
    if param.requires_grad:
        print(param.shape)

In [70]:
m.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(m.classifier[1].in_features, num_classes),
    nn.Softmax()
)
for param in m.classifier.parameters():
    if param.requires_grad:
        print(param.shape)

torch.Size([28, 1280])
torch.Size([28])


In [76]:
class ASLEfficientNet(pl.LightningModule):
    def __init__(self, num_classes):
        super(ASLEfficientNet, self).__init__()
        self.model = models.efficientnet_b0()
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.model.classifier[1].in_features, num_classes),
            nn.Softmax(dim=-1)
        )
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        acc = (outputs.argmax(dim=1) == labels).float().mean()
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        acc = (outputs.argmax(dim=1) == labels).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

In [77]:
# Initialize model
model = ASLEfficientNet(num_classes)


In [78]:
# Define callbacks
checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min", save_top_k=1, filename="best_model")

In [79]:
# Initialize Trainer
trainer = Trainer(max_epochs=20, callbacks=[checkpoint_callback])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
# Train model
trainer.fit(model, train_loader, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | EfficientNet     | 4.0 M  | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
16.174    Total estimated model params size (MB)
339       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:

# Save the final model
torch.save(model.state_dict(), "asl_efficientnet_model.pth")

In [ ]:
¨¨